# Faster R-CNN 구현
### 텐서플로우 기반
### 주석은 한글 위주로 작성

In [1]:
import numpy as np
import os
import glob
import cv2
import xmltodict
import tensorflow as tf
import math
import random
from tqdm import tqdm
from PIL import Image, ImageDraw

tf.compat.v1.enable_eager_execution()

## 훈련 이미지 가져오기

In [2]:
train_x_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/JPEGImages'
train_y_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/Annotations'

test_x_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/JPEGImages'
test_y_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/Annotations'

In [3]:
list_train_x = sorted([x for x in glob.glob(train_x_path + '/**')])    
list_train_y = sorted([x for x in glob.glob(train_y_path + '/**')]) 

list_test_x = sorted([x for x in glob.glob(test_x_path + '/**')])    
list_test_y = sorted([x for x in glob.glob(test_y_path + '/**')]) 

print(len(list_train_x))
print(len(list_test_x))

5011
4952


## 훈련용 데이터로 제작

In [4]:
# 앵커 테스트용
def get_image_224(image_file_path) :
    image = cv2.imread(image_file_path)
    image = cv2.resize(image, (224, 224))

    return image

In [5]:
# 입력용 이미지 생성. 224, 224로 변환시키고 채널 값(0~255)를 0~1 사이의 값으로 정규화 시켜줌
def make_input(image_file_list): 
    images_list = []
    
    for i in tqdm(range(0, len(image_file_list)), desc="get image") :
        
        image = cv2.imread(image_file_list[i])
        image = cv2.resize(image, (224, 224))/255
        
        images_list.append(image)
    
    return np.asarray(images_list)

In [6]:
# 존재하는 객체 종류를 알아내자
def get_Classes_inImage(xml_file_list):
    classes = []

    for xml_file_path in xml_file_list: 

        f = open(xml_file_path)
        xml_file = xmltodict.parse(f.read())
        # 사진에 객체가 여러개 있을 경우
        try: 
            for obj in xml_file['annotation']['object']:
                classes.append(obj['name'].lower()) # 들어있는 객체 종류를 알아낸다
        # 사진에 객체가 하나만 있을 경우
        except TypeError as e: 
            classes.append(xml_file['annotation']['object']['name'].lower()) 
        f.close()

    classes = list(set(classes)) # set은 중복된걸 다 제거하고 유니크한? 아무튼 하나만 가져온다. 그걸 리스트로 만든다
    classes.sort() # 정렬

    return classes

In [7]:
# 이미지 하나에 대한 라벨값(어떤 클래스가 있는지, 그 클래스는 어떤 박스를 갖고 있는지)
def get_label_fromImage(xml_file_path, Classes_inDataSet): # xml_file_path은 파일 하나의 경로를 나타낸다

    f = open(xml_file_path)
    xml_file = xmltodict.parse(f.read()) 

    # 우선 원래 이미지 크기를 얻는다. 왜냐하면 앵커는 224*224 기준으로 만들었는데 원본 이미지는 224*224가 아니기 때문.
    # 224*224에 맞게 줄일려고 하는거다
    Image_Height = float(xml_file['annotation']['size']['height'])
    Image_Width  = float(xml_file['annotation']['size']['width'])


    Classes_list = [] 
    Ground_Truth_Box_list = [] 
    class_label_list = [] # 원-핫 인코딩으로 만든 y값. 이건 Fast R-CNN 학습에 쓰인다.

    # multi-objects in image
    try:
        for obj in xml_file['annotation']['object']:
            obj_class = obj['name'].lower() 
            # 박스 좌표(왼쪽 위, 오른쪽 아래) 얻기
            x_min = float(obj['bndbox']['xmin']) 
            y_min = float(obj['bndbox']['ymin'])
            x_max = float(obj['bndbox']['xmax']) 
            y_max = float(obj['bndbox']['ymax'])

            # 224*224에 맞게 변형시켜줌
            x_min = float((224/Image_Width)*x_min)
            y_min = float((224/Image_Height)*y_min)
            x_max = float((224/Image_Width)*x_max)
            y_max = float((224/Image_Height)*y_max)

            Ground_Truth_Box = [x_min, y_min, x_max, y_max]

            index = Classes_inDataSet.index(obj_class) 

            Classes_list.append(index)
            Ground_Truth_Box_list.append(Ground_Truth_Box)

    # single-object in image
    except TypeError as e : 
        
        obj_class = xml_file['annotation']['object']['name'] 
        # 박스 좌표(왼쪽 위, 오른쪽 아래) 얻기
        x_min = float(xml_file['annotation']['object']['bndbox']['xmin']) 
        y_min = float(xml_file['annotation']['object']['bndbox']['ymin']) 
        x_max = float(xml_file['annotation']['object']['bndbox']['xmax']) 
        y_max = float(xml_file['annotation']['object']['bndbox']['ymax']) 

        # 224*224에 맞게 변형시켜줌
        x_min = float((224/Image_Width)*x_min)
        y_min = float((224/Image_Height)*y_min)
        x_max = float((224/Image_Width)*x_max)
        y_max = float((224/Image_Height)*y_max)

        Ground_Truth_Box = [x_min, y_min, x_max, y_max]  

        index = Classes_inDataSet.index(obj_class) 

        Classes_list.append(index)
        Ground_Truth_Box_list.append(Ground_Truth_Box)

    return Classes_list, Ground_Truth_Box_list # 각 이미지에 있는 클래스의 갯수는 다 다르기 때문에 리스트로 받는다. 리스트를 포함한 리스트 형태로 데이터를 만들 예정


In [8]:
# 파일 리스트
image_file_list = sorted([x for x in glob.glob(train_x_path + '/**')])
xml_file_list = sorted([x for x in glob.glob(train_y_path + '/**')])

## 데이터에 존재하는 클래스 종류 알아내기

In [9]:
Classes_inDataSet = get_Classes_inImage(xml_file_list)

In [10]:
len(Classes_inDataSet)

20

훈련용 이미지는 5011개, 테스트용 이미지는 4952개가 있음을 알 수 있다. 라벨값 역시 마찬가지.
<br>
이미지에 존재하는 클래스는 20종류다. 

## RPN - 앵커 준비
#### 입력 이미지 기준으로 앵커를 생성한다.
#### 풀링을 3번 하므로 2^3 = 8이니 (8, 8)부터 (16, 8)...등 8픽셀식 중심 좌표를 옮겨가며 앵커들을 k개씩 생성한다
#### 생성한 앵커들 중 사용할 가치가 있는 앵커를 걸러낸다(이미지 범위를 벗어나지 않는 앵커들만 선정)
#### 선정한 앵커 중 실제 물체의 box와 얼마나 곂치는지(IoU) 계산해본다. 확실히 겹친다 하는 애들을 Positive 앵커로, 거의 안겹친다 하는 애들은 Negataive 앵커로 선정한다. 애매한 애들은 거른다.
#### 이렇게 생성된 앵커들로 미니배치를 생성한다. Positive 128개, Negative 128개로 만드는게 ideal한 구성이긴 한데 Positive한 앵커가 별로 없다. 그래서 Positive를 128개 못채웠으면 Negataive 앵커로 채워준다. 

### 함수로 만드는 이유?
#### 이미지 별로 라벨에서 박스 좌표랑 객체 종류 뽑아내서 loss를 계산해야한다. 그래서 코드의 간결화?를 위해 함수를 만드는 것

In [11]:
# 앵커 생성 함수. 
# 얘가 좀 이상하다
def make_anchor(anchor_size, anchor_aspect_ratio) :
    # 입력 이미지(그래봤자 224*224긴 하지만)에 맞춰 앵커를 생성해보자 

    anchors = [] # [x,y,w,h]로 이루어진 리스트 
    anchors_state = [] # 이 앵커를 훈련에 쓸건가? 각 앵커별로 사용 여부를 나타낸다. 

    # 앵커 중심좌표 간격
    interval_x = 16
    interval_y = 16
    Center_max_x = 208 # 224 - 16, 중심좌표가 224가 될 수는 없다.
    Center_max_y = 208 # 224 - 16

    # 2단 while문 생성
    x = 8
    y = 8
    index_count = 0
    while(y <= 224): # 8~208 = 14개 
        while(x <= 224): # 8~208 = 14개 
            # k개의 앵커 생성. 여기서 k = len(anchor_size) * len(anchor_aspect_ratio)다
            for i in range(0, len(anchor_size)) : 
                for j in range(0, len(anchor_aspect_ratio)) :
                    anchor_width = anchor_aspect_ratio[j][0] * anchor_size[i]
                    anchor_height = anchor_aspect_ratio[j][1] * anchor_size[i]

                    anchor = [x, y, anchor_width, anchor_height]
                    anchors.append(anchor)
                    # 앵커가 이미지 경계선을 넘나드나? 필터링
                    if((x - (anchor_width/2) >= 0) and (y - (anchor_height/2) >= 0) and
                    (x + (anchor_width/2) <= 224) and (y + (anchor_height/2) <= 224)):
                        # 경계 안에 있으면 1
                        anchors_state.append(int(1))
                    else :
                        anchors_state.append(int(0))
            x = x + interval_x 
        y = y + interval_y
        x = 8
    return np.array(anchors), np.array(anchors_state) # 넘파이로 반환

In [12]:
anchor_size = [32, 64, 128] # 이미지 크기가 224*224라 32, 64, 128로 지정
anchor_aspect_ratio = [[1,1],[1,0.5], [0.5,1]] # W*L기준 
anchors, anchors_state = make_anchor(anchor_size, anchor_aspect_ratio) # 앵커 생성 + 유효한 앵커 인덱스 휙득

In [14]:
anchors_state[184]

1

#### Loss 사용을 위해 Class list도 만들자.
#### 논문의 loss 함수를 보면 pi가 있다. pi는 리스트인데 PASCAL VOC에 존재하는 객체들이 몇 %의 확률로 있느냐 나타내는거다
#### 이를 위해 어떤 객체들이 PASCAL VOC에 존재하는지 알아야한다. 

In [12]:
def get_iou(anchor, Ground_Truth_Box_List, Class_list) : # anchor, ground truth box list를 받아 각 IOU을 계산하고 제일 큰걸 반환 + 그 ground truth box가 가리키는 클래스 인덱스도 반환

    IoU_max = 0
    index_ground_truth_box = -1
    class_index = -1
    for i in range(0, len(Ground_Truth_Box_List)):

        ground_truth_box = Ground_Truth_Box_List[i]

        InterSection_min_x = max(anchor[0], ground_truth_box[0])
        InterSection_min_y = max(anchor[1], ground_truth_box[1])

        InterSection_max_x = min(anchor[2], ground_truth_box[2])
        InterSection_max_y = min(anchor[3], ground_truth_box[3])

        InterSection_Area = 0

        if (InterSection_max_x - InterSection_min_x + 1) >= 0 and (InterSection_max_y - InterSection_min_y + 1) >= 0 :
            InterSection_Area = (InterSection_max_x - InterSection_min_x + 1) * (InterSection_max_y - InterSection_min_y + 1)

        box1_area = (anchor[2] - anchor[0]) * (anchor[3] - anchor[1])
        box2_area = (ground_truth_box[2] - ground_truth_box[0]) * (ground_truth_box[3] - ground_truth_box[1])
        Union_Area = box1_area + box2_area - InterSection_Area

        IoU = (InterSection_Area/Union_Area)
        if IoU > IoU_max :
            IoU_max = IoU
            index_ground_truth_box = i
            class_index = Class_list[i]

    return IoU_max, index_ground_truth_box, class_index # 어떤 박스와 IoU가 제일 높았는지, 그 박스는 어떤 클래스를 나타내는지
    

## positive anchor, negative anchor

In [13]:
# 앵커들을 Positive, Negative 앵커로 나누고 각 앵커가 참고한 Ground Truth Box와 Class를 반환하자
def align_anchor(anchors, anchors_state, Ground_Truth_Box_list, Class_list):

    # 각 앵커는 해당 위치에서 구한 여러가지 Ground truth Box와의 ioU 중 제일 높은거만 가져온다. 
    IoU_List = np.array([])
    index_ground_truth_box_list = np.array([]) # 각 앵커가 어떤 Ground Truth Box를 보고 IoU를 계산했는가?
    index_class_list = np.array([]) # 각 앵커가 어떤 class를 봤는가?

    for i in range(0, len(anchors)):
        anchor_minX = anchors[i][0] - (anchors[i][2]/2)
        anchor_minY = anchors[i][1] - (anchors[i][3]/2)
        anchor_maxX = anchors[i][0] + (anchors[i][2]/2)
        anchor_maxY = anchors[i][1] + (anchors[i][3]/2)

        anchor = [anchor_minX, anchor_minY, anchor_maxX, anchor_maxY]
        IoU, index_ground_truth_box, class_index = get_iou(anchor, Ground_Truth_Box_list, Class_list)
        IoU_List = np.append(IoU_List, IoU)
        index_ground_truth_box_list = np.append(index_ground_truth_box_list, index_ground_truth_box)
        index_class_list = np.append(index_class_list, class_index)


    # positive, negative 앵커 분류
    for i in range(0, 14*14):
        # 각 위치에서 IoU가 가장 큰 앵커 or IoU가 0.7 넘는 앵커를 positive앵커로 한다. 

        # max 함수를 못믿겠다
        maxIoU_inOneSpot = 0
        for index in list(range(9*i, 9*i + 9)) :
            if IoU_List[index] > maxIoU_inOneSpot :
                maxIoU_inOneSpot = IoU_List[index]

        for num in range(0, 9):
            index = 9 * i + num
            if IoU_List[index] > 0.7 or (maxIoU_inOneSpot == IoU_List[index] and IoU_List[index] >= 0.3): # 조건을 좀 조정했다. 
                anchors_state[index] = 2
            elif IoU_List[index] < 0.3 : # negative anchor
                anchors_state[index] = 1
            else: # 애매한 앵커들
                anchors_state[index] = 0


    return anchors_state, index_ground_truth_box_list, index_class_list # 모든 앵커에 대한 ground truth box, class list

## 훈련을 위한 미니배치 256(positive 128, negative 128개) 선별
### loss 안에서 실시한다

In [14]:
def get_minibatch_index(output):
    # 2개의 loss함수에서 불리는 함수. 우선 cls_layer의 loss에서 불린 뒤 reg_layer의 loss에서 불릴거다. 
    # 그러니 우선 cls에서 부를 때 미니배치 인덱스를 선별해 반환하고 그 다음 reg에서 부를 때는 선별된 인덱스 리스트를 반환 후 초기화한다.

    index_list = np.array([])

    if len(get_minibatch_index.index_list) == 0:
        get_minibatch_index.index_list = np.zeros(14*14*9) # 각 앵커가 미니배치 뽑혔나 안뽑혔나
        index_pos = np.array([])
        index_neg = np.array([])
        # cls_layer_output을 보고 긍정, 부정 앵커 분류. 그렇게 데이터셋을 구성함
        for i in range(0, 1764):
            if output[i][0] == 1.0 : # positive anchor
                index_pos = np.append(index_pos, i)
            elif output[i][0] == 0.0 : # negative anchor
                index_neg = np.append(index_neg, i)

        max_for = min([128, len(index_pos)])
        ran_list = random.sample(range(0, len(index_pos)), max_for)

        for i in range(0, len(ran_list)) :
            index = int(index_pos[ran_list[i]])
            get_minibatch_index.index_list[index] = 1

        ran_list = random.sample(range(0, len(index_neg)), 256 - max_for) # 랜덤성 증가?를 위해 또다시 난수 생성
        for i in range(0, len(ran_list)) :
            index = int(index_neg[ran_list[i]])
            get_minibatch_index.index_list[index] = 1

        index_list = get_minibatch_index.index_list

    else : # 데이터가 이미 존재
        index_list = get_minibatch_index.index_list
        get_minibatch_index.index_list = np.array([])

    return index_list
get_minibatch_index.index_list = np.array([])

## Loss 계산을 위한 함수들

In [15]:
def Smooth_L1(ti, ti_star) :
    difference_ti = ti - ti_star
    smooth_L1 = 0
    if abs(difference_ti) < 1: smooth_L1 = 0.5 * (difference_ti * difference_ti)
    else : smooth_L1 = abs(difference_ti) - 0.5

    return smooth_L1

In [16]:
# Loss_Regression. Lreg에 해당
def Loss_Regression(predict_box, anchor_box, groundTruth_box) : 
    groundTruth_box = np.array([(groundTruth_box[2] + groundTruth_box[0])/2, (groundTruth_box[1] + groundTruth_box[3])/2, groundTruth_box[2] - groundTruth_box[0], groundTruth_box[3] - groundTruth_box[1]])

    t_x = (predict_box[0] - anchor_box[0])/anchor_box[2]
    t_y = (predict_box[1] - anchor_box[1])/anchor_box[3]
    t_w = math.log10(predict_box[2]/anchor_box[2])
    t_h = math.log10(predict_box[3]/anchor_box[3])


    t_x_star = (groundTruth_box[0] - anchor_box[0])/anchor_box[2]
    t_y_star = (groundTruth_box[1] - anchor_box[1])/anchor_box[3]
    t_w_star = math.log10(groundTruth_box[2]/anchor_box[2])
    t_h_star = math.log10(groundTruth_box[3]/anchor_box[3])

    # Smooth L1 구하기
    # 구성요소가 4개니까 4번 구해야겠지?
    smooth_L1_x = Smooth_L1(t_x, t_x_star)
    smooth_L1_y = Smooth_L1(t_y, t_y_star)
    smooth_L1_w = Smooth_L1(t_w, t_w_star)
    smooth_L1_h = Smooth_L1(t_h, t_h_star)

    smooth_L1_list = smooth_L1_x + smooth_L1_y + smooth_L1_w + smooth_L1_h

    return smooth_L1_list # 모아서 반환


In [17]:
# L_cls
def Loss_Classes(cls_layer_output, pi_ground_truth) :
    log_loss = -pi_ground_truth * math.log10(cls_layer_output[0]) - (1-pi_ground_truth)*math.log10(1-cls_layer_output[0])
    return log_loss

In [18]:
def loss_rpn_cls(y_true, y_pred): 
    # 60k -> 60*9 = 540개의 미니배치 훈련률과 20k -> 20*9 = 180개의 미니배치 훈련률이 다르다
    lr_coefficient = 0
    if loss_rpn_cls.counter <= 540 :
        lr_coefficient = 1
    else :
        lr_coefficient = 0.1

    
    cls_layer_output = y_pred.numpy()[0]
    cls_layer_output_label = y_true.numpy()[0]

    cls_layer_output = np.reshape(cls_layer_output, (-1, 2))
    cls_layer_output_label = np.reshape(cls_layer_output_label, (-1, 2))

    minibatch_index_list = get_minibatch_index(cls_layer_output_label)
    cls_layer_output_minibatch = np.array([])
    cls_layer_output_label_minibatch = np.array([])

    # 미니배치 선별
    for i in range(0, len(minibatch_index_list)) :  
        if minibatch_index_list[i] == 1 : # 미니배치로 뽑혔으면 
            score = [cls_layer_output[i][0]  , cls_layer_output[i][1]] # 오브젝트다 vs 아니다 2개 항목에 대한 스코어 저장(각 앵커당)
            score = tf.nn.softmax(score)
            cls_layer_output_minibatch = np.append(cls_layer_output_minibatch, score)
            cls_layer_output_label_minibatch = np.append(cls_layer_output_label_minibatch, cls_layer_output_label[i])

    cls_layer_output_minibatch = np.reshape(cls_layer_output_minibatch, (-1, 2))
    cls_layer_output_label_minibatch = np.reshape(cls_layer_output_label_minibatch, (-1, 2))

    Lcls_sum = 0
    # loss 계산
    for i in range(0, 256):
        pi_star = cls_layer_output_label_minibatch[i][0] # postive면 1, negative면 0
        Lcls_sum = Lcls_sum + Loss_Classes(cls_layer_output_minibatch[i], pi_star)

    loss = (1/256) * Lcls_sum
    loss = loss * lr_coefficient
    
    loss_rpn_cls.counter = loss_rpn_cls.counter + 1

    if loss_rpn_cls.counter > 80*9 :
        loss_rpn_cls.counter = 0

    return loss


loss_rpn_cls.counter = 0 # 미니배치 한번 돌아갈 때마다 1씩 올라감

In [18]:
def loss_rpn_reg(y_true, y_pred): 
    lr_coefficient = 0
    if loss_rpn_reg.counter <= 540 :
        lr_coefficient = 1
    else :
        lr_coefficient = 0.1

    lambda_forLoss = 10
    N_reg = 14*14

    reg_layer_output = y_pred.numpy()[0]
    reg_layer_output_label = y_true.numpy()[0]


    # 예측값 가공
    reg_layer_output = np.reshape(reg_layer_output, (14,14,36))

    for y in range(0, 14):
        for x in range(0, 14):
                for i in range(0, len(anchor_size)):
                    for j in range(0, len(anchor_aspect_ratio)):

                        center_x = 8 + 16 * x 
                        center_y = 8 + 16 * y
                        w = anchor_size[i] * anchor_aspect_ratio[j][0]
                        h = anchor_size[i] * anchor_aspect_ratio[j][1]

                        # 원래 출력값은 각 위치 앵커에서 변화율로 사용한다. 
                        reg_layer_output[y][x][4*(3*i+j)] = center_x + reg_layer_output[y][x][4*(3*i+j)]
                        reg_layer_output[y][x][4*(3*i+j) + 1] = center_y + reg_layer_output[y][x][4*(3*i+j) + 1]
                        reg_layer_output[y][x][4*(3*i+j) + 2] = w + reg_layer_output[y][x][4*(3*i+j) + 2]
                        reg_layer_output[y][x][4*(3*i+j) + 3] = h + reg_layer_output[y][x][4*(3*i+j) + 3]


    # (-1, 4)로 변환 -> 각 앵커별 (x,y,w,h)
    reg_layer_output = np.reshape(reg_layer_output, (-1, 4))
    reg_layer_output_label = np.reshape(reg_layer_output_label, (-1, 4))

    minibatch_index_list = get_minibatch_index(reg_layer_output_label)
    reg_layer_output_minibatch = np.array([])
    reg_layer_output_label_minibatch = np.array([])
    anchors_minibatch = np.array([])


    # 미니배치 선별
    for i in range(0, len(minibatch_index_list)) : 
        if minibatch_index_list[i] == 1 : # 미니배치로 뽑혔으면 
            reg_layer_output_minibatch = np.append(reg_layer_output_minibatch, reg_layer_output[i])
            reg_layer_output_label_minibatch = np.append(reg_layer_output_label_minibatch, reg_layer_output_label[i])
            anchors_minibatch = np.append(anchors_minibatch, anchors[i])

    # 미니배치 reshape
    reg_layer_output_minibatch = np.reshape(reg_layer_output_minibatch, (-1, 4))
    reg_layer_output_label_minibatch = np.reshape(reg_layer_output_label_minibatch, (-1, 4))
    anchors_minibatch = np.reshape(anchors_minibatch, (-1, 4))

    Lreg_sum = 0
    # loss 계산
    for i in range(0, 256):
        # x,y,w,h 합이 0이 아니면 로스에 포함
        if sum(reg_layer_output_label_minibatch[i]):
            Lreg_sum = Lreg_sum + Loss_Regression(reg_layer_output_minibatch[i], anchors_minibatch[i], reg_layer_output_label_minibatch[i])


    loss = lambda_forLoss*(1/N_reg)*Lreg_sum
    loss = loss * lr_coefficient
    
    loss_rpn_reg.counter = loss_rpn_cls.counter + 1

    if loss_rpn_reg.counter > 80*9 :
        loss_rpn_reg.counter = 0

    return loss


loss_rpn_reg.counter = 0 # 미니배치 한번 돌아갈 때마다 1씩 올라감

## 데이터셋 생성

In [21]:
def make_dataset_forRPN(image_file_list, xml_file_list, Classes_inDataSet, anchors, anchors_state) :
    image_list = make_input(image_file_list)

    valid_minibatch_anchor_list = np.array([])
    cls_layer_label_list = np.array([])
    reg_layer_label_list = np.array([])

    index_ground_truth_boxes_list = np.array([])
    index_classes_list = np.array([])

    Classes_labels_list = []
    Grond_Truth_Boxes_list = []

    for i in tqdm(range(0, len(xml_file_list)), desc="get label"):
        class_label_list, Ground_Truth_Box_list = get_label_fromImage(xml_file_list[i], Classes_inDataSet) 
        anchors_state, index_ground_truth_box_list, index_class_list = align_anchor(anchors, anchors_state, Ground_Truth_Box_list, class_label_list) 
        # pos, neg 앵커 뭔지 들어있는 state, 각 앵커가 어떤 ground truth box를 참조했는지, 각 앵커가 어떤 종류의 클래스를 참조했는지
        
        # '한 이미지'에 있는 Ground Truth Box에 대한 정보
        Classes_labels_list.append(class_label_list)
        Grond_Truth_Boxes_list.append(Ground_Truth_Box_list)

        cls_layer_label = np.array([]) # cls_layer는 14*14*18이므로 그에 맞게 가공
        reg_layer_label = np.array([]) # 14*14*36

        for j in range(0, len(anchors_state)) :
            if anchors_state[j] == 2 : # positive
                ground_box = Ground_Truth_Box_list[int(index_ground_truth_box_list[j])] # positive 앵커가 참고한 Ground Truth Box
                #RPN
                cls_layer_label = np.append(cls_layer_label, np.array([1.0,0.0])) # 해당 앵커의 output이 [1,0] -> positive
                reg_layer_label = np.append(reg_layer_label, ground_box)
            elif anchors_state[j] == 1 : # negative
                cls_layer_label = np.append(cls_layer_label, np.array([0.0,1.0])) # 해당 앵커 output이 [0,1] -> negative
                reg_layer_label = np.append(reg_layer_label, np.array([0,0,0,0]))
            else : 
                cls_layer_label = np.append(cls_layer_label, np.array([0.5,0.5])) # 해당 앵커 output이 [0.5, 0.5] -> 무의미한 값
                reg_layer_label = np.append(reg_layer_label, np.array([0,0,0,0]))

        index_classes_list = np.append(index_classes_list, index_class_list) # 각 앵커마다 어떤 클래스를 참조했는지 -> 1764개씩 쌓임
        index_ground_truth_boxes_list = np.append(index_ground_truth_boxes_list, index_ground_truth_box_list) # 각 이미지별 모든 앵커에 대한 Ground Truth Box를 기록 -> 5011 * 1764

        cls_layer_label_list = np.append(cls_layer_label_list, cls_layer_label) # (1764 * 2)개씩 입력
        reg_layer_label_list = np.append(reg_layer_label_list, reg_layer_label) # (1764 * 4)개씩 입력
    
    index_classes_list = np.reshape(index_classes_list, (-1,1764))
    index_ground_truth_boxes_list = np.reshape(index_ground_truth_boxes_list, (-1, 1764))
    cls_layer_label_list = np.reshape(cls_layer_label_list, (-1, 14,14,18))
    reg_layer_label_list = np.reshape(reg_layer_label_list, (-1, 14,14,36))



    return image_list, cls_layer_label_list, reg_layer_label_list, Classes_labels_list, Grond_Truth_Boxes_list, index_classes_list
    # 입력 데이터셋, (RPN)이 앵커가 Postive인가 Negative인가 Useless인가, 이 앵커가 어떤 Ground Truth Box를 보고 IoU를 계산했는가, 각 이미지에 어떤 객체들이 들어있나, 그 객체들의 박스 좌표는 어디인가, 각 이미지의 앵커들이 참조한 클래스는 어떤건가

## RPN과 Detector 두 모델에 쓰일 데이터셋 생성

In [23]:
anchor_size = [32, 64, 128] # 이미지 크기가 224*224라 32, 64, 128로 지정
anchor_aspect_ratio = [[1,1],[1,0.5], [0.5,1]] # W*L기준 
anchors, anchors_state = make_anchor(anchor_size, anchor_aspect_ratio) # 앵커 생성 + 유효한 앵커 인덱스 휙득

image_list, cls_layer_label_list, reg_layer_label_list, Classes_labels_list, Grond_Truth_Boxes_list, index_classes_list = make_dataset_forRPN(image_file_list, xml_file_list, Classes_inDataSet, anchors, anchors_state)

get label: 100%|██████████| 5011/5011 [27:44<00:00,  3.01it/s]


In [25]:
print("image_list.shape : ", image_list.shape)
print("cls_layer_label_list.shape : ", cls_layer_label_list.shape)
print("reg_layer_label_list.shape : ", reg_layer_label_list.shape)
print("Classes_labels_list_len : ", len(Classes_labels_list))
print("Grond_Truth_Boxes_list_len : ", len(Grond_Truth_Boxes_list))
print("index_classes_list.shape : ", index_classes_list.shape)

image_list.shape :  (5011, 224, 224, 3)
cls_layer_label_list.shape :  (5011, 14, 14, 18)
reg_layer_label_list.shape :  (5011, 14, 14, 36)
Classes_labels_list_len :  5011
Grond_Truth_Boxes_list_len :  5011
index_classes_list.shape :  (5011, 1764)


In [41]:
int((32)/16)

2

In [40]:
int((32)/16)

2

In [34]:
Grond_Truth_Boxes_list[4]

[[61.700598802395206, 32.256, 204.5508982035928, 211.904]]

### RPN 생성

In [25]:
class RPN(tf.keras.Model):

  def __init__(self, initializer, regularizer, shared_convNet):
    super(RPN, self).__init__(name='rpn')
    # 레이어 

    # 공용 레이어
    self.conv1_1 = SharedConvNet.layers[0]
    self.conv1_2 = SharedConvNet.layers[1]
    self.pooling_1 = SharedConvNet.layers[2]
    self.conv2_1 = SharedConvNet.layers[3]
    self.conv2_2 = SharedConvNet.layers[4]
    self.pooling_2 = SharedConvNet.layers[5]
    self.conv3_1 = SharedConvNet.layers[6]
    self.conv3_2 = SharedConvNet.layers[7]
    self.conv3_3 = SharedConvNet.layers[8]
    self.pooling_3 = SharedConvNet.layers[9]
    self.conv4_1 = SharedConvNet.layers[10]
    self.conv4_2 = SharedConvNet.layers[11]
    self.conv4_3 = SharedConvNet.layers[12]
    self.pooling_4 = SharedConvNet.layers[13]
    self.conv5_1 = SharedConvNet.layers[14]
    self.conv5_2 = SharedConvNet.layers[15]
    self.conv5_3 = SharedConvNet.layers[16]
    # RPN만의 레이어
    self.intermediate_layer = tf.keras.layers.Conv2D(512, (3, 3), padding = 'SAME' , name = "intermediate_layer", input_shape = (14,14,512))
    self.cls_Layer = tf.keras.layers.Conv2D(18, (1, 1), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, name = "output_1")
    self.reg_layer = tf.keras.layers.Conv2D(36, (1, 1), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, name = "output_2")
    
  def call(self, inputs):
    # 정방향 연산
    output1_1 = self.conv1_1(inputs)
    output1_2 = self.conv1_2(output1_1)
    output1_pooling = self.pooling_1(output1_2)
    output2_1 = self.conv2_1(output1_pooling)
    output2_2 = self.conv2_2(output2_1)
    output2_pooling = self.pooling_2(output2_2)
    output3_1 = self.conv3_1(output2_pooling)
    output3_2 = self.conv3_2(output3_1)
    output3_3 = self.conv3_3(output3_2)
    output3_pooling = self.pooling_3(output3_3)
    output4_1 = self.conv4_1(output3_pooling)
    output4_2 = self.conv4_2(output4_1)
    output4_3 = self.conv4_3(output4_2)
    output4_pooling = self.pooling_4(output4_3)
    output5_1 = self.conv5_1(output4_pooling)
    output5_2 = self.conv5_2(output5_1)
    output5_3 = self.conv5_3(output5_2)
    # RPN
    feature_map = self.intermediate_layer(output5_3)
    cls_Layer_output = self.cls_Layer(feature_map)
    reg_Layer_output = self.reg_layer(feature_map)

    return cls_Layer_output, reg_Layer_output # 4k, 2k개 아웃풋 반환

## RPN 훈련

In [26]:
def training_RPN(model, image_list, reg_layer_label_list, cls_layer_label_list, traning_step, EPOCH):
    
    if traning_step == 1 : # 처음에는 conv3_1까지만 훈련
        model.conv1_1.trainable = False
        model.conv1_2.trainable = False
        model.conv2_1.trainable = False
        model.conv2_2.trainable = False
    elif traning_step == 3 : # 두번 째, 그러니까 training step == 3일 때는 sharedConvLayer를 고정
        model.conv1_1.trainable = False
        model.conv1_2.trainable = False
        model.conv2_1.trainable = False
        model.conv2_2.trainable = False
        model.conv3_1.trainable = False
        model.conv3_2.trainable = False
        model.conv3_3.trainable = False
        model.conv4_1.trainable = False
        model.conv4_2.trainable = False
        model.conv4_3.trainable = False
        model.conv5_1.trainable = False
        model.conv5_2.trainable = False
        model.conv5_3.trainable = False       

    losses = {'output_1' : loss_rpn_cls, 'output_2' : loss_rpn_reg}

    Optimizer = tf.compat.v1.train.RMSPropOptimizer(learning_rate=0.001, decay = 0.0005,momentum=0.9,epsilon=1e-10) # momentum = 0.9
    model.compile(optimizer=Optimizer, loss=losses, run_eagerly=True)

    history = model.fit(np.asarray(image_list), [cls_layer_label_list, reg_layer_label_list], batch_size = 1, epochs = EPOCH)

    return model, history


## RPN 모델 생성

In [27]:
max_num = len(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers) # 레이어 최대 개수

SharedConvNet = tf.keras.models.Sequential()
for i in range(0, max_num-1):
    SharedConvNet.add(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers[i])

initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
regularizer = tf.keras.regularizers.l2(0.0005)

RPN_Model = RPN(initializer, regularizer, SharedConvNet)
RPN_Model.run_eagerly = True # 모델 내부 함수에서 쉽게 연산할 수 있게 설정

## RPN 훈련

In [36]:
RPN_Model, history = training_RPN(RPN_Model, image_list, reg_layer_label_list, cls_layer_label_list, 1, 1)

5011/5011 [==============================] - 2649s 528ms/step - loss: 0.3247 - output_1_loss: 0.2486 - output_2_loss: 0.0761


### RPN에서 값 얻어내기

In [ ]:
def Process_output(cls_layer_output, reg_layer_output):

    for y in range(0, 14):
        for x in range(0, 14):
                for i in range(0, len(anchor_size)):

                    for j in range(0, len(anchor_aspect_ratio)):
                        score = [cls_layer_output[y][x][2*(3*i+j)]  , cls_layer_output[i][2*(3*i+j) + 1]] # 오브젝트다 vs 아니다 2개 항목에 대한 스코어 저장(각 앵커당)
                        score = tf.nn.softmax(score)
                        cls_layer_output[y][x][2*(3*i+j)] = score[0]
                        cls_layer_output[y][x][2*(3*i+j) + 1] = score[1]
                        
                        center_x = 8 + 16 * x 
                        center_y = 8 + 16 * y
                        w = anchor_size[i] * anchor_aspect_ratio[j][0]
                        h = anchor_size[i] * anchor_aspect_ratio[j][1]

                        # 원래 출력값은 각 위치 앵커에서 변화율로 사용한다. 
                        reg_layer_output[y][x][4*(3*i+j)] = center_x + reg_layer_output[y][x][4*(3*i+j)]
                        reg_layer_output[y][x][4*(3*i+j) + 1] = center_y + reg_layer_output[y][x][4*(3*i+j) + 1]
                        reg_layer_output[y][x][4*(3*i+j) + 2] = w + reg_layer_output[y][x][4*(3*i+j) + 2]
                        reg_layer_output[y][x][4*(3*i+j) + 3] = h + reg_layer_output[y][x][4*(3*i+j) + 3]

    cls_layer_output = np.reshape(cls_layer_output, (-1,2))
    reg_layer_output = np.reshape(reg_layer_output, (-1,4))

    return cls_layer_output, reg_layer_output


In [ ]:
def Get_Output_RPN(RPN_Model, image_list) :
    cls_layer_output_list = np.array([])
    reg_layer_output_list = np.array([])
    for i in tqdm(range(0, len(image_list)), desc = "get_output"):
        cls_layer_output, reg_layer_output = RPN_Model(np.expand_dims(image_list[i], axis=0))

        cls_layer_output = cls_layer_output[0].numpy()
        reg_layer_output = reg_layer_output[0].numpy()

        # 가공하자
        cls_layer_output, reg_layer_output = Process_output(reg_layer_output, cls_layer_output)

        cls_layer_output_list = np.append(cls_layer_output_list, cls_layer_output)
        reg_layer_output_list = np.append(reg_layer_output_list, reg_layer_output)
    
    cls_layer_output_list = np.reshape(cls_layer_output_list, (-1,1764,2))
    reg_layer_output_list = np.reshape(reg_layer_output_list, (-1,1764,4))

    return cls_layer_output_list, reg_layer_output_list

In [41]:
cls_layer_output_list, reg_layer_output_list = Get_Output_RPN(RPN_Model, image_list)

get_output: 100%|██████████| 5011/5011 [23:02<00:00,  3.63it/s]


## Fast R-CNN

### 입력값 : 공유 특성맵 + RoI
### 출력값 : 예측한 객체 종류, 객체의 좌표(r, c, h, w). (r,c)는 왼쪽위 좌표고 h,w는 너비, 높이

In [ ]:
# Detector 구조
# 입력 이미지 -> 특성맵.
# 특성맵 + RoI -> RoI위치만 추출. 그러면 나한테 있는 데이터는 RoI 위치별 특성맵, 각 RoI의 Truth Box, Classes
# Roi 특성맵을 연산을 통해 cls, loc 휙득


In [65]:
def nms(cls_layer_output, reg_layer_output, reg_layer_label, index_classes): # 한 이미지에서 뽑아낸 RPN의 출력값 2개, 이미지에 있는 모든 앵커들이 참조한 Ground Truth Box, 그 Ground Truth Box의 Class
    nms_RoI_List = [] # 각 이미지마다 RoI 갯수가 다르다. 그래서 리스트로 저장한다. 리스트 안에 리스트 저장하는 방식으로 len = 5011인 리스트를 생성할거다 
    nms_GroundTruthBox_List = [] # NMS에 들어간 RoI가 참조한 Ground Truth Box 
    nms_Classes_List = []
    for i in range(0, len(cls_layer_output)) :
        if cls_layer_output[i][0] > 0.7 : # 해당 앵커의 object score가 0.7을 넘겼으면
            # 저장
            nms_RoI_List.append(reg_layer_output[i]) # RoI
            nms_GroundTruthBox_List.append(reg_layer_output_list[i]) # Ground Truth Box
            # 그 Box가 가리키는 클래스 인덱스
            nms_Classes_List.append(index_classes[i]



    return nms_RoI_List, nms_GroundTruthBox_List, nms_Classes_List # 선별한 애들을 반환

In [ ]:
def get_nms_list(cls_layer_output_list, reg_layer_output_list, reg_layer_label_list, index_classes_list) :
    # RPN output 2개, 이미지에서 각 앵커(1764)들이 어떤 Ground Truth Box보고 IoU 계산했는지, 이미지에서 각 앵커(1764)들이 어떤 '클래스'의 Ground Truth Box보고 IoU 계산했는지
    # Detector 훈련에 필요한 데이터를 얻는 곳이다
    
    NMS_RoIs_List = [] # 전체 입력 이미지의 RoI를 이미지별로 저장(리스트 안에 리스트)
    NMS_GroundTruthBoxes_List = []
    NMS_Classes_List = []

    for i in tqdm(range(0, len(cls_layer_output_list)), desc = "get_RoI"): # 5011개에 대한 nms 구한다
        nms_RoI_List, nms_GroundTruthBox_List, nms_Classes_List = nms(cls_layer_output_list[i], reg_layer_output_list[i], reg_layer_label_list[i], index_classes_list[i])
        NMS_RoIs_List.append(nms_RoI_List)
        NMS_GroundTruthBoxes_List.append(NMS_GroundTruthBoxes_List)
        NMS_Classes_List.append(nms_Classes_List)

    return NMS_RoIs_List, NMS_GroundTruthBoxes_List, NMS_Classes_List

In [63]:
def change_RoI(nms_RoI_List) :
    # RoI_list : (-1, 4) <- (x,y,w,h)
    RoI_list_forDetector = np.array([])
    for i in range(0, len(RoI_list)) :
        r = RoI_list[i][0] - (RoI_list[i][2]/2)
        c = RoI_list[i][1] - (RoI_list[i][3]/2)
        w = RoI_list[i][2]
        h = RoI_list[i][3]

        roi_forDetector = np.array([r,c,w,h])
        RoI_list_forDetector = np.append(RoI_list_forDetector, roi_forDetector)
    RoI_list_forDetector = np.reshape(RoI_list_forDetector, (-1,4)) 

    return RoI_list_forDetector

## Detector 훈련에 필요한 데이터셋 휙득

In [ ]:
NMS_RoIs_List, NMS_GroundTruthBoxes_List, NMS_Classes_List = get_nms_list(cls_layer_output_list, reg_layer_output_list, reg_layer_label_list, index_classes_list)

## Detector 모델 생성

In [ ]:
class Detector(tf.keras.Model):

  def __init__(self, initializer, regularizer, shared_convNet):
    super(Detector, self).__init__(name='rpn')
    # 레이어 
    # 공용 레이어
    self.conv1_1 = SharedConvNet.layers[0]
    self.conv1_2 = SharedConvNet.layers[1]
    self.pooling_1 = SharedConvNet.layers[2]
    self.conv2_1 = SharedConvNet.layers[3]
    self.conv2_2 = SharedConvNet.layers[4]
    self.pooling_2 = SharedConvNet.layers[5]
    self.conv3_1 = SharedConvNet.layers[6]
    self.conv3_2 = SharedConvNet.layers[7]
    self.conv3_3 = SharedConvNet.layers[8]
    self.pooling_3 = SharedConvNet.layers[9]
    self.conv4_1 = SharedConvNet.layers[10]
    self.conv4_2 = SharedConvNet.layers[11]
    self.conv4_3 = SharedConvNet.layers[12]
    self.pooling_4 = SharedConvNet.layers[13]
    self.conv5_1 = SharedConvNet.layers[14]
    self.conv5_2 = SharedConvNet.layers[15]
    self.conv5_3 = SharedConvNet.layers[16]

    Classify_layer_initializer tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
    Box_regression_layer_initializer tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.001, seed=None)

    # RoI Pooling : H*W(7*7)에 맞게 입력 특성맵을 pooling. RoI에 해당하는 영역을 7*7로 Pooling한다. 
    self.RoIPoolingLayer = tf.keras.layers # 7*7로 만들어버린다. 
    self.Flatten_layer = tf.keras.layers.Flatten() # 여기서 4096개가 되어야한다.
    self.Classify_layer = tf.keras.layers.Dense(21, activation='softmax', kernel_initializer = Classify_layer_initializer, name = "output_1")
    self.reg_layer = tf.keras.layers.Conv2D(84, activation= None, kernel_initializer = Box_regression_layer_initializer, name = "output_2")
    
  def call(self, inputs): # input으로 리스트를 넣으면 된다.

    Input_Image = inputs[0]
    RoI_List = inputs[1]

    # 정방향 연산
    output1_1 = self.conv1_1(Input_Image)
    output1_2 = self.conv1_2(output1_1)
    output1_pooling = self.pooling_1(output1_2)
    output2_1 = self.conv2_1(output1_pooling)
    output2_2 = self.conv2_2(output2_1)
    output2_pooling = self.pooling_2(output2_2)
    output3_1 = self.conv3_1(output2_pooling)
    output3_2 = self.conv3_2(output3_1)
    output3_3 = self.conv3_3(output3_2)
    output3_pooling = self.pooling_3(output3_3)
    output4_1 = self.conv4_1(output3_pooling)
    output4_2 = self.conv4_2(output4_1)
    output4_3 = self.conv4_3(output4_2)
    output4_pooling = self.pooling_4(output4_3)
    output5_1 = self.conv5_1(output4_pooling)
    output5_2 = self.conv5_2(output5_1)
    output5_3 = self.conv5_3(output5_2)
    # Detector
    # Pooling


    return cls_Layer_output, reg_Layer_output # 4k, 2k개 아웃풋 반환

## Loss 함수

In [ ]:
def loss_detector_loc(y_true, y_pred):
    # y_true : 해당 이미지 내에 있는 모든 클래스의 
    return loss

## 모델 생성

In [ ]:
max_num = len(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers) # 레이어 최대 개수

SharedConvNet = tf.keras.models.Sequential()
for i in range(0, max_num-1):
    SharedConvNet.add(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers[i])

Detector_Model = Detector(SharedConvNet)
RPN_Model.run_eagerly = True # 모델 내부 함수에서 쉽게 연산할 수 있게 설정

In [ ]:
def training_Detector(model, , traning_step, EPOCH):

    losses = {'output_1' : 'categorical_crossentropy', 'output_2' : loss_detector_loc}

    model.compile(optimizer=Optimizer, loss=losses, run_eagerly=True)

    history = model.fit(np.asarray(image_list), [cls_layer_label_list, reg_layer_label_list], batch_size = 1, epochs = EPOCH)

    return model, history
